## Exercise 3
In the videos you looked at how you would improve Fashion MNIST using Convolutions. For your exercise see if you can improve MNIST to 99.8% accuracy or more using only a single convolutional layer and a single MaxPooling 2D. You should stop training once the accuracy goes above this amount. It should happen in less than 20 epochs, so it's ok to hard code the number of epochs for training, but your training must end once it hits the above metric. If it doesn't, then you'll need to redesign your layers.

When 99.8% accuracy has been hit, you should print out the string "Reached 99.8% accuracy so cancelling training!"


In [1]:
!nvidia-smi

Fri Oct  8 04:47:33 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.74       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P8    28W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [6]:
import tensorflow as tf
from os import path, getcwd, chdir

path = f"{getcwd()}/tmp/mnist.npz"

In [7]:
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)


In [8]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epochs, logs={}):
        threshold = .998
        if logs.get('accuracy') is not None and logs.get('accuracy') >= threshold:
            print(f"Reached 99.8% accuracy so cancelling training!")
            self.model.stop_training = True

In [9]:
def train_mnist_conv():
    callback = myCallback()

    mnist = tf.keras.datasets.mnist
    (training_images, training_labels), (test_images, test_labels) = mnist.load_data(path=path)
    training_images = training_images.reshape(60000, 28, 28, 1)
    training_images = training_images / 255.0
    test_images = test_images.reshape(10000, 28, 28, 1)
    test_images = test_images / 255.0

    model = tf.keras.models.Sequential([
            tf.keras.layers.Conv2D(64, (3, 3), activation='relu', input_shape=(28, 28, 1)),
            tf.keras.layers.MaxPool2D(2, 2),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(256, activation=tf.nn.relu),
            tf.keras.layers.Dense(10, activation=tf.nn.softmax)
    ])

    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    # model fitting
    history = model.fit(training_images, training_labels, epochs=20, callbacks=[callback], verbose=2)
    # model fitting
    return history.epoch, history.history['accuracy'][-1]

In [10]:
_, _ = train_mnist_conv()

Epoch 1/20
1875/1875 - 42s - loss: 0.1247 - accuracy: 0.9625
Epoch 2/20
1875/1875 - 17s - loss: 0.0405 - accuracy: 0.9871
Epoch 3/20
1875/1875 - 13s - loss: 0.0232 - accuracy: 0.9924
Epoch 4/20
1875/1875 - 14s - loss: 0.0153 - accuracy: 0.9951
Epoch 5/20
1875/1875 - 14s - loss: 0.0097 - accuracy: 0.9967
Epoch 6/20
1875/1875 - 14s - loss: 0.0083 - accuracy: 0.9971
Epoch 7/20
1875/1875 - 14s - loss: 0.0059 - accuracy: 0.9980
Reached 99.8% accuracy so cancelling training!
